# Demo of Opening up an OFX file and categorising the transactions therein ready for upload to YNAB 4

Put the full path to your OFX file in the cell below this one.

Done:
- Parse an OFX file into a Pandas dataframe
- Categorisee the transactions - but you'll see the transaction categories are hard coded balow.  They are only for quick demo purposes - tweak them and add to your heart's content.  

Todo:
- push the data out to a CSV in a format that YNAB can ingest
- QIF and QFX parsing.  CSV import
- loop over a directory of ofx files
- some sort of interface for transaction categorisation: possibly parsing a spreadsheet filled out by the user?
    - future: maybe use scikit learn to auto categorise transations based on historic YNAB data??
- Transaction deduplication (or could just rely on YNAB to do that?)
    
Tested with 
- a multi-account OFX file exported from Natwest bank (nwolb.com).  
- Python 3.9.1,
- Pandas 1.2.0 (must update that ...)
- matplotlib 3.5.0 etc
- running on Ubuntu (yes yes I know that's weird because YNAB is Windows software, it's just that's where I've got Python running, OK?
- Dependency on OFX parse - see https://pypiorg/project/ofxparse/  You'll probably need to install via pip or conda - eg https://anaconda.org/conda-forge/ofxparse
    
 ## Part 1 - ingest the OFX file

In [ ]:
## PUT PATH TO YOUR OFX BANK EXPORT IN HERE:
ofxpath = r"/tmp/ynabstuff/Bank-download-20200915.ofx"

import os, pandas as pd, hashlib as hl
from ofxparse import OfxParser
import matplotlib.pyplot as plt 
import datetime as dt

pd.plotting.register_matplotlib_converters()
%matplotlib inline

# NB re importing categories in to YNAB4:  "This worked fine in YNAB 4 (using column name "Category" and values formatted as "Category Group: Category")" 
# - this is from https://support.youneedabudget.com/t/x1bw4k/how-to-import-categories-from-csv
#  However the forum is shortly to close...
#
# From the YNAB website, sample CSV import:
#
# Date,Payee,Memo,Outflow,Inflow
# 06/22/21,Payee 1,Memo,100.00,
# 06/23/21,Payee 2,Memo,,500.00

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print (f"Ready at {dt.datetime.now()}")


In [ ]:
with open (ofxpath, 'rb') as f:
    
    # MD5 of the file
    file_md5 = hl.md5(f.read()).hexdigest()
    print (f"Using {ofxpath}; file md5 hash is {file_md5}.")
    
    # let ofxparse do its thing
    ofx = OfxParser.parse(f)
    
    # need this list in a minute:
    dfs = []
    
    # for every account in this ofx
    for act in ofx.accounts:
        
        # make a dataframe of the transactions in the account statement
        testtrans = pd.DataFrame ( [vars(t) for t in act.statement.transactions] )
        
        # add on some account columns (could do more?)
        testtrans ['sort_code'] = act.routing_number
        testtrans ['account_num'] = act.account_id
        testtrans ['ofx']  = ofxpath
        testtrans ['ofx_md5'] = file_md5
        
        # add this dataframe to the list of dataframes
        dfs.append(testtrans)

    # make a dataframe combining transactions from all accounts in the list into one ledger:
    output = pd.concat (dfs)
print (f"\nTen random transactions from the OFX file (total tranactions is {output.amount.count()}):")
output.sample (n=10)

## Part 2 - Categorise the transactions

In [ ]:

# and now let's categorise some transactions

output['category'] = 'TBC'

# Obviously these are hard-coded transaction types!  Could be reading these in from json or similar; amend / add to your heart's conent!
def categorise (trans):
    return \
        'Income' if trans.amount > 0 \
        else 'Mortgage' if trans.payee == 'YOUR MORTGAGE PAYEE HERE' \
        else 'Holiday' if trans.payee in [ 
             'BRITISH A', 'PAYPAL *AIRBNB'  ,
            ]  \
        else 'Supermarket' if (
            any (
                word in trans.payee for word in 
                ['SAINS', 'TESCO', "ALDI", "MARKS", "CO-OP"]
            )
            or trans.payee in ['MY LOCAL FARM SHOP']
        ) \
        else 'DIY' if (
            trans.payee.count("WICKES")
        ) \
        else 'Amazon/Paypal - Could be absolutely anything' if (
            trans.payee.count("AMAZON")
            or trans.payee.count("AMZN")  
            or trans.payee.count("PAYPAL")  
        ) \
        else 'CASH' if trans.payee == 'CASH' \
        else 'Bills' if trans.payee in [
            "COUNCIL TAX", "ELECTRICITY COMPANY" 'TV LICENCE MBP'
        ] \
        else 'Insurance' if trans.payee in [
            "YOUR", "INSURERS", "HERE"
        ] \
        else 'Childcare' if trans.payee in ['NURSERY PAYYEE HERE'] \
        else 'Phone / Broadband' if (
            trans.payee.count("VIRGIN")
            or trans.payee.count("VODA")
        ) \
        else 'Restaurants' if (
            any (
                word in trans.payee for word in [
                "RESTAURANT", "PIZZA EXPRESS", "WAGAMAMA", 'JUST EAT'] #etc
            )
            or trans.payee in ['THE WHITE HORSE', 'LOCH FYNE'] 
        ) \
        else 'Clothes' if trans.payee.count ("CLARKS")\
        else 'Lunch, Coffee' if trans.payee in ['PRET A MANGER', 'COSTA'] \
        else "Car, fuel" if (trans.payee.count ("SHELL")
                             or trans.payee.count ("ESSO")
                            )\
        else trans.category    

output['category'] =  output.apply (lambda op: categorise(op), axis = 1)

print ("3 displays of remaining uncategorised transactions:")
print ("a. The largest individual transactions")
display (  
    output[output['category'] == 'TBC'] [['amount', 'memo', 'payee']] \
        .sort_values(by=['amount'], ascending=True).head(10)
)
print ("b. by payees by spending")
display (  
    output[output['category'] == 'TBC'] [['amount', 'payee']].groupby ('payee')\
    ['amount'].count().sort_values ( ascending = False).head (10)
)
print ("c. payees by number of  transactions")
display ( 
    output[output['category'] == 'TBC'] [['amount', 'payee']].groupby ('payee')\
    ['amount'].sum().sort_values ( ascending = True).head (10)
)


# Right, now we can make something which starts to look a bit like the YNAB budget sheet:
output['month_year'] = pd.to_datetime(output['date']).dt.to_period('M')
output['amount'] = output.amount.astype(float)

print ("Approximation of YNAB budget view:")
pv = output.pivot_table ( 
       index = ["category"], 
       # values = ['Charged Hours/Quantity','Labor Cost'], 
       values = ['amount'], 
       aggfunc = "sum" , 
       margins = 1, 
       columns = ["month_year"], 
       fill_value = '' # otherwise it uses NaN
)

display (pv)

In [ ]:
pv_plt =  output.pivot_table ( 
       columns = "category", 
       # values = ['Charged Hours/Quantity','Labor Cost'], 
       values = 'amount', 
       aggfunc = "sum" , 
       margins = 0, 
       index = "month_year", 
       fill_value = 0 # otherwise it uses NaN
)

display (pv_plt)
pv_plt.plot(
    kind='bar', 
    legend = True, 
    stacked = True, 
    figsize = (25,10),
    rot = 45,
)